In [ ]:
#danaderp May6'19
#Prediction For Main Issues Data Set

In [ ]:
import csv
from tensorflow.keras.preprocessing import text
from nltk.corpus import gutenberg
from string import punctuation
from tensorflow.keras.preprocessing.sequence import skipgrams

In [ ]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
pd.options.display.max_colwidth = 200
%matplotlib inline

In [ ]:
from nltk.stem.snowball import SnowballStemmer
englishStemmer=SnowballStemmer("english")

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.layers import Dot, Input, Dense, Reshape, LSTM, Conv2D, Flatten, MaxPooling1D, Dropout, MaxPooling2D
from tensorflow.keras.layers import Embedding, Multiply, Subtract
from tensorflow.keras.models import Sequential, Model
from tensorflow.python.keras.layers import Lambda
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint

In [ ]:
# visualize model structure
#from IPython.display import SVG
#from keras.utils.vis_utils import model_to_dot
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.manifold import TSNE

In [ ]:
from utils.read_data import Dynamic_Dataset, Processing_Dataset
from utils.vectorize_sentence import Embeddings

In [ ]:
path = "../data/augmented_dataset/"
process_unit = Processing_Dataset(path)
ground_truth = process_unit.get_ground_truth()

In [ ]:
dataset = Dynamic_Dataset(ground_truth, path,False)

In [ ]:
test, train = process_unit.get_test_and_training(ground_truth, isZip = True)

In [ ]:
print(len(test))
print(len(train))
print(test[0])
print(train[0])

11612
104510
('(1,0)', b'The North American Ismaili Games (aka hr.apps.n166983741) application 5.26.2 for Android does not verify X.509 certificates from SSL servers, which allows man-in-the-middle attackers to spoof servers and obtain sensitive information via a crafted certificate.')
('(1,0)', b'The currently used Rails version, in the stable branch, is insecure\n\nYou should update the Gemfile.lock to hotfix this.\n\nhttp://weblog.rubyonrails.org/2014/2/18/Rails_3_2_17_4_0_3_and_4_1_0_beta2_have_been_released/')


In [ ]:
#Train/Test split verification
#for elem in test:
#    print(elem[0])

In [ ]:
#Preprocesing Corpora
embeddings = Embeddings()
max_words = 5000 #<------- [Parameter]
pre_corpora_train = [doc for doc in train if len(doc[1])< max_words]
pre_corpora_test = [doc for doc in test if len(doc[1])< max_words]

KeyboardInterrupt: 

In [ ]:
print(len(pre_corpora_train))
print(len(pre_corpora_test))

In [ ]:
embed_path = 'datasets/word_embeddings-embed_size_100-epochs_100.csv'
embeddings_dict = embeddings.get_embeddings_dict(embed_path)

In [ ]:
corpora_train = [embeddings.vectorize(doc[1], embeddings_dict) for doc in pre_corpora_train]#vectorization Inputs
corpora_test = [embeddings.vectorize(doc[1], embeddings_dict) for doc in pre_corpora_test]#vectorization

In [ ]:
target_train = [[int(list(doc[0])[1]),int(list(doc[0])[3])] for doc in pre_corpora_train]#vectorization Output
target_test = [[int(list(doc[0])[1]),int(list(doc[0])[3])]for doc in pre_corpora_test]#vectorization Output
#target_train

In [ ]:
max_len_sentences_train = max([len(doc) for doc in corpora_train]) #<------- [Parameter]
max_len_sentences_test = max([len(doc) for doc in corpora_test]) #<------- [Parameter]

In [ ]:
max_len_sentences = max(max_len_sentences_train,max_len_sentences_test)
print("Max. Sentence # words:",max_len_sentences)

In [ ]:
min_len_sentences_train = min([len(doc) for doc in corpora_train]) #<------- [Parameter]
min_len_sentences_test = min([len(doc) for doc in corpora_test]) #<------- [Parameter]

In [ ]:
min_len_sentences = max(min_len_sentences_train,min_len_sentences_test)
print("Mix. Sentence # words:",min_len_sentences)

In [ ]:
embed_size = np.size(corpora_train[0][0])

In [ ]:
#BaseLine Architecture <-------
embeddigs_cols = embed_size
input_sh = (max_len_sentences,embeddigs_cols,1)
#Selecting filters? 
#https://stackoverflow.com/questions/48243360/how-to-determine-the-filter-parameter-in-the-keras-conv2d-function
#https://stats.stackexchange.com/questions/196646/what-is-the-significance-of-the-number-of-convolution-filters-in-a-convolutional

N_filters = 128 # <-------- [HyperParameter] Powers of 2 Numer of Features
K = 2 # <-------- [HyperParameter] Number of Classess

In [ ]:
input_sh

In [ ]:
#baseline_model = Sequential()
gram_input = Input(shape = input_sh)

In [ ]:
# 1st Convolutional Layer
conv_1_layer = Conv2D(filters=64, input_shape=input_sh, activation='relu', 
                      kernel_size=(7,7), strides=(2,2), padding='valid')(gram_input)

In [ ]:
conv_1_layer.shape

In [ ]:
# Max Pooling
max_1_pooling = MaxPooling2D(pool_size=(3,3), strides=None, padding='valid')(conv_1_layer)

In [ ]:
max_1_pooling.shape

In [ ]:
# 2nd Convolutional Layer
conv_2_layer = Conv2D(filters=256, kernel_size=(5,5), activation='relu', 
                      strides=(1,1), padding='valid')(max_1_pooling)

In [ ]:
conv_2_layer.shape

In [ ]:
# Max Pooling
max_2_pooling = MaxPooling2D(pool_size=(3,3), strides=None, padding='valid')(conv_2_layer)

In [ ]:
max_2_pooling.shape

In [ ]:
# 3rd Convolutional Layer
conv_3_layer =  Conv2D(filters=512, kernel_size=(3,3), activation='relu', 
                      strides=(1,1), padding='valid')(max_2_pooling)

In [ ]:
conv_3_layer.shape

In [ ]:
# 4th Convolutional Layer
conv_4_layer = Conv2D(filters=512, kernel_size=(3,1), activation='relu', 
                      strides=(1,1), padding='valid')(conv_3_layer)

In [ ]:
conv_4_layer.shape

In [ ]:
# 5th Convolutional Layer
conv_5_layer = Conv2D(filters=256, kernel_size=(3,1), activation='relu', 
                      strides=(1,1), padding='valid')(conv_4_layer)

In [ ]:
conv_5_layer.shape

In [ ]:
# Max Pooling
max_5_pooling = MaxPooling2D(pool_size=(3,1), strides=None, padding='valid')(conv_5_layer)

In [ ]:
max_5_pooling.shape

In [ ]:
# Fully Connected layer
fully_connected = Flatten()(max_5_pooling)

In [ ]:
fully_connected.shape

In [ ]:
# 1st Fully Connected Layer
deep_dense_1_layer = Dense(1024, activation='relu')(fully_connected)
deep_dense_1_layer = Dropout(0.4)(deep_dense_1_layer) # <-------- [HyperParameter]

In [ ]:
deep_dense_1_layer.shape

In [ ]:
# 2nd Fully Connected Layer
deep_dense_2_layer = Dense(1024, activation='relu')(deep_dense_1_layer)
deep_dense_2_layer = Dropout(0.4)(deep_dense_2_layer) # <-------- [HyperParameter]

In [ ]:
deep_dense_2_layer.shape

In [ ]:
# 3rd Fully Connected Layer
deep_dense_3_layer = Dense(128, activation='relu')(deep_dense_2_layer)
deep_dense_3_layer = Dropout(0.4)(deep_dense_3_layer) # <-------- [HyperParameter]

In [ ]:
deep_dense_3_layer.shape

In [ ]:
predictions = Dense(K, activation='softmax')(deep_dense_3_layer)

In [ ]:
#Criticality Model
criticality_network = Model(inputs=[gram_input],outputs=[predictions]) 

In [ ]:
print(criticality_network.summary())

In [ ]:
#Seting up the Model
criticality_network.compile(optimizer='adam',loss='binary_crossentropy',
                                  metrics=['accuracy'])

In [ ]:
#Data set organization
from tempfile import mkdtemp
import os.path as path

In [ ]:
#Memoization 
file_corpora_train_x = path.join(mkdtemp(), 'alex-res_temp_corpora_train_x.dat') #Update per experiment
file_corpora_test_x = path.join(mkdtemp(), 'alex-res_temp_corpora_test_x.dat')

In [ ]:
#Shaping
shape_train_x = (len(corpora_train),max_len_sentences,embeddigs_cols,1)
shape_test_x = (len(corpora_test),max_len_sentences,embeddigs_cols,1)

In [ ]:
#Data sets
corpora_train_x = np.memmap(
        filename = file_corpora_train_x, 
        dtype='float32', 
        mode='w+', 
        shape = shape_train_x)

In [ ]:
corpora_test_x = np.memmap( #Test Corpora (for future evaluation)
        filename = file_corpora_test_x, 
        dtype='float32', 
        mode='w+', 
        shape = shape_test_x)

In [ ]:
target_train_y = np.array(target_train) #Train Target
target_test_y = np.array(target_test) #Test Target (for future evaluation)

In [ ]:
corpora_train_x.shape

In [ ]:
target_train_y.shape

In [ ]:
corpora_test_x.shape

In [ ]:
target_test_y.shape

In [ ]:
#Reshaping Train Inputs
for doc in range(len(corpora_train)):
    #print(corpora_train[doc].shape[1])
    for words_rows in range(corpora_train[doc].shape[0]):
        embed_flatten = np.array(corpora_train[doc][words_rows]).flatten() #<--- Capture doc and word
        for embedding_cols in range(embed_flatten.shape[0]):
            corpora_train_x[doc,words_rows,embedding_cols,0] = embed_flatten[embedding_cols]

In [ ]:
#Reshaping Test Inputs (for future evaluation)
for doc in range(len(corpora_test)):
    for words_rows in range(corpora_test[doc].shape[0]):
        embed_flatten = np.array(corpora_test[doc][words_rows]).flatten() #<--- Capture doc and word
        for embedding_cols in range(embed_flatten.shape[0]):
            corpora_test_x[doc,words_rows,embedding_cols,0] = embed_flatten[embedding_cols]

In [ ]:
#CheckPoints
#csv_logger = CSVLogger(system+'_training.log')
filepath = "alex-res/embeds100-improvement-{epoch:02d}-{val_accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath)
callbacks_list = [checkpoint]

In [ ]:
#Model Fitting
history = criticality_network.fit(
            x = corpora_train_x, 
            y = target_train_y,
            #batch_size=64,
            epochs=20, #5 <------ Hyperparameter
            validation_split = 0.2
            #callbacks=callbacks_list
)

In [ ]:
#Saving Training History
df_history = pd.DataFrame.from_dict(history.history)
df_history.to_csv('alex-res/history_training.csv', encoding='utf-8',index=False)

In [ ]:
criticality_network.save(filepath)
df_history.head()

In [ ]:
#Saving Test Data
np.save('alex-res/corpora_test_x.npy',corpora_test_x)
np.save('alex-res/target_test_y.npy',target_test_y)

In [ ]:
#Evaluation
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
 
epochs2 = range(len(acc))
 
plt.plot(epochs2, acc, 'b', label='Training')
plt.plot(epochs2, val_acc, 'r', label='Validation')
plt.title('Training and validation accuracy')
plt.ylabel('acc')
plt.xlabel('epoch')
plt.legend()
 
plt.figure()
 
plt.plot(epochs2, loss, 'b', label='Training')
plt.plot(epochs2, val_loss, 'r', label='Validation')
plt.title('Training and validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend()
 
plt.show()